In [ ]:
#import os
#os.chdir('/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/featureExtraction/')
%cd ../featureSelection/

In [1]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML
from sklearn import metrics
import import_ipynb
from eng_feature_selection import *

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,6 days 23 hours 21 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,"14 days, 14 hours and 41 minutes"
H2O cluster name:,H2O_from_python_mackenzie_dm3nzl
H2O cluster total nodes:,1
H2O cluster free memory:,1.219 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [25]:
X_train = h2o.H2OFrame(getUnivariateData())
y_train = h2o.H2OFrame(getLabels())
X_test = h2o.H2OFrame(getUnivariateTestData())
y_test = h2o.H2OFrame(getTestLabels()) 


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = X_train.cbind(y_train)
test = X_test.cbind(y_test)


In [13]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [14]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=10) #max_models=10 or 20?, seed?
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [15]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20190703_143453,0.379212,0.352589,0.3191,0.101825
StackedEnsemble_AllModels_AutoML_20190703_143453,0.380945,0.351513,0.31898,0.101748
GLM_grid_1_AutoML_20190703_143453_model_1,0.384846,0.347966,0.319396,0.102014
GBM_1_AutoML_20190703_143453,0.38608,0.343009,0.318445,0.101407
GBM_2_AutoML_20190703_143453,0.3924,0.343937,0.318668,0.101549
XGBoost_1_AutoML_20190703_143453,0.394923,0.338059,0.316211,0.0999894
GBM_3_AutoML_20190703_143453,0.395216,0.347392,0.319843,0.102299
XGBoost_2_AutoML_20190703_143453,0.397681,0.344188,0.321341,0.10326
GBM_4_AutoML_20190703_143453,0.401665,0.352246,0.319949,0.102368
XGBoost_3_AutoML_20190703_143453,0.410509,0.35616,0.328232,0.107736


In [16]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190703_143453
No model summary for this model


ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.04370837949664802
RMSE: 0.20906549092724036

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.10182483318575941
RMSE: 0.31910003632992495


In [17]:
preds = aml.predict(test)
print(preds)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1,p2
2,0.107316,0.170745,0.721938
2,0.0450257,0.0502032,0.904771
2,0.16064,0.226383,0.612977
2,0.0877273,0.122501,0.789772
2,0.0917272,0.131997,0.776276
2,0.22616,0.116045,0.657795
2,0.0590925,0.0577924,0.883115
2,0.0954635,0.126629,0.777908
2,0.107068,0.218627,0.674305
2,0.0656908,0.0806016,0.853708


In [18]:
var = preds["predict"].cbind(test[y])
print(var)

predict,labels
2,1
2,1
2,2
2,1
2,1
2,1
2,1
2,1
2,1
2,0


In [21]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

[[   9    9  268]
 [  28   51 3759]
 [   4   64  765]]
0.16643130925963284
